<center><h1>@decorators</h1></center>
  
Greg Lutostanski

[github.com/lutostag](https://github.com/lutostag)

Software Developer
    
  
![Canonical](https://assets.ubuntu.com/v1/5d6da5c4-logo-canonical-aubergine.svg)

# Decorators
* wrappers for functions, classes, or methods
* more functionality
* defined as functions or classes
* called with the "@" symbol

In [1]:
def decorator(func):
    return func

In [2]:
@decorator
def method():
    pass

In [3]:
def multiply_by_three(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) * 3
    return wrapper

In [4]:
@multiply_by_three
def fake_add(a, b):
    return a + b

print(fake_add(1, 2))

9


In [5]:
# same as
def fake_add(a, b):
    return a + b

fake_add = multiply_by_three(fake_add)

print(fake_add(1, 2))

9


So we know a bit of how they work.

What are some of the uses?
* retries
* cache
* logging

locking, deprecating, disabling

In [6]:
def multiply_by_three(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) * 3
    return wrapper


@multiply_by_three
def fake_add(a, b):
    return a + b

In [7]:
# what about arguments to the decorator?

def multiply_by(number):
    def outer_wrapper(func):
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs) * number
        return wrapper
    return outer_wrapper

@multiply_by(4)
def fake_add(a, b):
    return a + b

print(fake_add(1, 3))

16


* property
* staticmethod
* classmethod
* functools.lru_cache
* functools.total_ordering
* unittest.mock.patch
* asyncio.coroutine
* contextlib.contextmanager